In [ ]:
%load_ext autoreload
%autoreload 2

In [19]:
#basics
import sys,os

#utils
import paths

#main libraries
import pandas as pd

#model Libraries
from sklearn import utils
from sklearn import preprocessing

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GroupKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

In [3]:
#private modules 
#data
from src.data import make_dataset

#utils
from src.utils import memory_usage

#model
from src.models import train_model
from src.models import evaluation

In [25]:
%%time
df_training,features,target = make_dataset.get_data(nrows=50000, low_memory=False, dataset="training")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 314 entries, id to target_kazutsugi
dtypes: float64(311), int32(1), object(2)
memory usage: 119.6+ MB
None
CPU times: user 3.63 s, sys: 689 ms, total: 4.32 s
Wall time: 11.3 s


In [5]:
%%time
#df_tournament,features,target = make_dataset.get_data(nrows=5000, low_memory=False, dataset="tournament")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [6]:
#encoding target to multiclass
lab_enc = preprocessing.LabelEncoder()
encoded_target_train = lab_enc.fit_transform(df_training[target])
#encoded_target_tournament = lab_enc.fit_transform(df_tournament[target])

In [39]:
memory_usage.memory()

Available memory (%): 24.791336059570312


In [8]:
results, res_cv = list(), dict()

In [38]:
rf_param_grid =  {
    'rf__n_estimators' : [10],
}


#encoding target to multiclass
lab_enc = preprocessing.LabelEncoder()
encoded_target_train = lab_enc.fit_transform(df_training[target])


#Group K-fold
CV = GroupKFold(n_splits = 3)
grp = list(CV.split(X = df_training[features], y = df_training[target],  groups = df_training.era.values))


#scorer function
scorer = make_scorer(roc_auc_score, multi_class='ovr', needs_proba=True)

result = evaluation.evaluate_model(df_training[features], encoded_target_train,
                                   RandomForestClassifier(), 'rf',
                                   rf_param_grid, 
                                   scorer, 
                                   n_iter=1, 
                                   cv_folds=grp,
                                   pipeline=None)

results.append(result)
res_cv[result[1]] = evaluation.get_cv_scores(result[0])

==> Starting K-fold cross validation for rf model, 50000 examples
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    7.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    7.8s finished


==> Elapsed seconds: 11.933
Best rf model: RandomForestClassifier(n_estimators=10)
Best rf score (val): 0.532


In [40]:
rf_param_grid =  {
    #'rf__bootstrap' : [True, False],
    #'rf__ccp_alpha' : 0.0, 
    #'rf__class_weight': [None, {0:1,1:1}, {0:1,1:10}, {0:1,1:100}, 'balanced'],
    #'rf__criterion' : 'gini', 
    #'rf__max_depth' : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 
    #'rf__max_features' : ['auto', 'sqrt'],
    #'rf__max_leaf_nodes' : None,
    #'rf__max_samples' : None,
    #'rf__min_impurity_decrease' : 0.0, 
    #'rf__min_impurity_split' : None,
    #'rf__min_samples_leaf' : [1, 2, 4],
    #'rf__min_samples_split' : [2, 5, 10],
    #'rf__min_weight_fraction_leaf' : 0.0, 
    'rf__n_estimators' : [10, 20],#, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
    #'rf__oob_score' : False, 
    #'rf__random_state' : None,
    #'rf__verbose' : 0, 
    #'rf__warm_start' : False
}


scorer = make_scorer(mean_squared_error)

result = evaluation.evaluate_model(df_training[features], encoded_target_train,
                                   RandomForestRegressor(), 'rf',
                                   rf_param_grid, 
                                   scorer, n_iter=2, 
                                   cv_folds=grp, pipeline=None)

results.append(result)
res_cv[result[1]] = evaluation.get_cv_scores(result[0])

==> Starting K-fold cross validation for rf model, 50000 examples
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  1.3min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.8min finished


==> Elapsed seconds: 229.635
Best rf model: RandomForestRegressor(n_estimators=10)
Best rf score (val): 2.273


In [11]:
model_result = list(filter(lambda x: x[1] == 'rf', results))[0]
model = model_result[0]

In [13]:
%%time
from joblib import parallel_backend

with parallel_backend('threading', n_jobs=-1):
    # Train the model
    model = RandomForestClassifier(n_estimators=10, n_jobs=-1)
    model.fit(df_training[features], encoded_target_train)
    # Test the model
    #y_pred = clf.predict(X_test)


CPU times: user 416 ms, sys: 20.6 ms, total: 436 ms
Wall time: 171 ms


In [ ]:
#### ALL ABOUT NUMEROX e metricas